<a href="https://colab.research.google.com/github/semoennaciri/Tipemohamedennaciri/blob/main/stroke_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#stroke prediction 

##Importation de la data 

In [ ]:
from google.colab import files 
import io 
uploaded = files.upload()

KeyboardInterrupt: ignored

### libraries

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
import scipy.stats as stats
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold, learning_curve, train_test_split
from sklearn.metrics import fbeta_score, make_scorer
from sklearn import metrics 
from imblearn.ensemble import BalancedBaggingClassifier
sns.set_style("whitegrid")
import os
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer


##Data

In [ ]:
#loading data
data=pd.read_excel('healthcare-dataset-stroke-data.xlsx')
# drop id
data.drop("id", axis=1, inplace=True)
data

In [ ]:
data.describe()

##Correlation


In [ ]:
mat=data.corr()
mat

##data visualisation 

In [ ]:
import plotly.express as px

df = px.data.tips()
fig = px.scatter(data, x="age", y="stroke", facet_col="stroke",
                 width=800, height=400)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)

fig.show()

In [ ]:
import plotly.express as px

fig = px.histogram(data, x="heart_disease",y='stroke',color='heart_disease',height=400)
fig.show()

In [ ]:
import plotly.express as px
fig = px.histogram(data, x="ever_married",y='stroke',color='ever_married',height=400)
fig.show()

In [ ]:
import plotly.express as px
fig = px.histogram(data, x="work_type",y='stroke',color='work_type',height=400)
fig.show()

In [ ]:
import plotly.express as px
fig = px.histogram(data, x="smoking_status",y='stroke',color='smoking_status',height=400)
fig.show()

In [ ]:
import plotly.express as px
fig = px.histogram(data, x="Residence_type",y='stroke',color='Residence_type',height=400)
fig.show()

In [ ]:
data.head(2)

In [ ]:
import plotly.express as px
fig = px.histogram(data, x="stroke",color='stroke',height=400)
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter(x=data['avg_glucose_level'], y=data['stroke'])
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter(x=data['bmi'], y=data['stroke'])
fig.show()

##imbalanced data 

In [ ]:
lg = LogisticRegression(random_state = 34)
lg.fit(X_train, y_train)
y_pred = lg.predict(X_test)

In [ ]:
print('Train Score is : ' , lg.score(X_train, y_train))
print('Test Score is : ' , lg.score(X_test, y_test))

##data cleaning 

In [ ]:
# drop outliers
numerical = ["age", "bmi", "avg_glucose_level"]
for col in numerical:
    col_zscore = col + "_zscore"
    data[col_zscore] = (data[col] - data[col].mean())/data[col].std(ddof=0)
    data[col + "_outlier"] = (abs(data[col+"_zscore"])>3).astype(int)
    data_drop = data[(data[col + "_outlier"]==1)]
    data = pd.concat([data, data_drop]).drop_duplicates(keep=False)   
    data.drop(col_zscore, axis=1, inplace=True)
    data.drop(col + "_outlier", axis=1, inplace=True)

# Gender - drop "other" -> drop 1 record with unique category
data.drop(data[data.gender == 'Other'].index, axis = 0, inplace=True)


In [ ]:
# train-test split
X = data.drop('stroke', axis=1)
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=34)

In [ ]:
# Categorical variable encoding
nominals = [c for c in X_train if X_train[c].dtypes=="object"]
for c in nominals:
    label_encoder = LabelEncoder()
    label_encoder.fit(list(X_train[c]) + list(X_test[c]))
    X_train[c] = label_encoder.transform(X_train[c])
    X_test[c] = label_encoder.transform(X_test[c])

In [ ]:
# standartizing the data - mean = 0 ,sd=1
continious = [c for c in X_train if X_train[c].dtypes=="float64"]
sc = StandardScaler()
train_transformed = sc.fit_transform(X_train[continious])
X_train[continious] = pd.DataFrame(train_transformed,columns = X_train[continious].columns, index=X_train[continious].index)
X_test[continious] = pd.DataFrame(sc.transform(X_test[continious]),columns = X_test[continious].columns, index=X_test[continious].index)

# can't drop missing data in the bmi column, it contains 40 stroke cases (16% of all stroke cases we have)
# using KNN-Imputer
imputer = KNNImputer()
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns, index=X_test.index)

###understanding the problem

In [ ]:
pos = data[data.stroke==1].shape[0]
neg = data[data.stroke==0].shape[0]
print("Stroke: " + str(pos) + ", No Stroke: " + str(neg), ", ratio: " + str(pos/neg) + "\n")
sns.catplot(data=data, x="stroke", kind="count", palette="winter_r", alpha=.6)

In [ ]:
from sklearn.metrics import confusion_matrix
lg = LogisticRegression(random_state = 34)
lg.fit(X_train, y_train)
y_pred = lg.predict(X_test)
confusion_matrix(y_test, y_pred)

##résolution du probleme

###under-samling

In [ ]:
from sklearn.utils import resample

# splitting the classes
train = X_train.copy()
train['stroke'] = y_train
stroke = train[(train.stroke == 1)]
not_stroke = train[(train.stroke == 0)]

# n_sample parameter - match the minority class, sample() shuffles the rows
not_stroke_downsampled = resample(not_stroke, replace=False, n_samples=len(stroke), random_state=34)
train_downsampled = pd.concat([not_stroke_downsampled, stroke]).sample(frac=1)
train_downsampled.stroke.value_counts()

In [ ]:
downsampled_y = train_downsampled['stroke']
downsampled_X = train_downsampled.drop('stroke', axis=1)

###log regression after under-samling

In [ ]:
lg = LogisticRegression(random_state = 34)
lg.fit(downsampled_X, downsampled_y)
y_pred = lg.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print('Train Score is : ' , lg.score(X_train, y_train))
print('Test Score is : ' , lg.score(X_test, y_test))

### over-samling

In [ ]:
# n_sample parameter - match the minority class, sample() shuffles the rows
stroke_upsampled = resample(stroke, replace=True, n_samples=len(not_stroke), random_state=34)
train_upsampled = pd.concat([stroke_upsampled, not_stroke]).sample(frac=1)
train_upsampled.stroke.value_counts()

In [ ]:
upsampled_y = train_upsampled['stroke']
upsampled_X = train_upsampled.drop('stroke', axis=1)

###log regression after over-samling

In [ ]:
lg = LogisticRegression(random_state = 34)
lg.fit(upsampled_X, upsampled_y)
y_pred = lg.predict(X_test)
confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

##Simple support vector classifier


In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=34)
synthetic_X, synthetic_y = sm.fit_resample(X_train, y_train)
synthetic_y.value_counts()

In [ ]:
kfold = RepeatedStratifiedKFold(n_splits=7, n_repeats=3)
f2 = make_scorer(fbeta_score, beta=2)

In [ ]:
svc = SVC(probability=True, random_state=34)
print("Cross validation score average: \n")
print(cross_val_score(svc, synthetic_X, y = synthetic_y, scoring = f2, cv =kfold, n_jobs=-1).mean())
svc.fit(synthetic_X,synthetic_y)
y_pred_proba_svc = svc.predict_proba(X_test)[::,1]
y_pred_svc = svc.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred_svc)